## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

2023-11-13 15:56:34.284414: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading the data

In [6]:
TRAIN_DIR = "./imgs/train/"
img_height = 480
img_width = 640
batch_size = 32
training_dataset = tf.keras.utils.image_dataset_from_directory('./imgs/train/',
                                                      validation_split=0.2,
                                                        subset="training",
                                                        seed=123,
                                                        image_size=(img_height, img_width),
                                                        batch_size=batch_size)
validation_dataset = tf.keras.utils.image_dataset_from_directory('./imgs/train/',
                                                      validation_split=0.2,
                                                        subset="validation",
                                                        seed=123,
                                                        image_size=(img_height, img_width),
                                                        batch_size=batch_size)

Found 22424 files belonging to 10 classes.
Using 17940 files for training.
Found 22424 files belonging to 10 classes.
Using 4484 files for validation.


In [3]:
class_names = training_dataset.class_names

In [4]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
training_dataset = training_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

## A basic Keras model

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=1
history = model.fit(
  training_dataset,
  validation_data=validation_dataset,
  epochs=epochs
) # I get around 87% acc and val_acc with 1 epoch - around 8 minutes of training

## Transfer Learning with MobileNet

In [5]:
num_classes = len(class_names)
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [9]:
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=True,
                                               weights='imagenet')

14536120/14536120 [==============================] - 4s 0us/step


In [10]:
IMG_SIZE = (480, 640)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   include_top=False, # <== Important!!!!
                                                   weights='imagenet')

9406464/9406464 [==============================] - 3s 0us/step


In [11]:
# freeze the base model by making it non trainable
base_model.trainable = False

In [12]:
inputs = tf.keras.Input(shape=IMG_SHAPE)

In [13]:
x = preprocess_input(inputs)

In [14]:
x = base_model(x, training=False)

In [15]:
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [16]:
x = tf.keras.layers.Dropout(0.2)(x)

In [17]:
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

In [18]:
model = tf.keras.Model(inputs, outputs)

In [19]:
base_learning_rate = 0.001

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [23]:
initial_epochs = 1
model.fit(training_dataset, validation_data=validation_dataset, epochs=initial_epochs)

561/561 [==============================] - ETA: 0s - loss: 1.6320 - accuracy: 0.5298

2023-11-13 16:21:20.020562: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [4484]
	 [[{{node Placeholder/_4}}]]
2023-11-13 16:21:20.020782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [4484]
	 [[{{node Placeholder/_4}}]]


561/561 [==============================] - 900s 2s/step - loss: 1.6320 - accuracy: 0.5298 - val_loss: 1.1304 - val_accuracy: 0.8330


### Fine tuning

In [31]:
base_model = model.layers[3]
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  154


In [32]:
fine_tune_at = 120

In [33]:
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [34]:
# Define a SparseCategoricalCrossentropy loss function. Use from_logits=True
loss_function=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1 * base_learning_rate)
metrics=['accuracy']

In [39]:
model.compile(loss=loss_function,
              optimizer = optimizer,
              metrics=metrics)

In [ ]:
fine_tune_epochs = 1
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(training_dataset,
                         epochs=total_epochs,
                         validation_data=validation_dataset)

My doubt: why don't we use base_model.fit instead of model.fit? Didn't we made our freezings on base_model?